In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, kpss
%matplotlib inline

### Data

In [ ]:
df_orig = pd.read_csv('data/daily-minimum-temperatures.csv')
df_orig['date'] =  pd.to_datetime(df_orig['date'], format='%Y-%m-%d')
df = df_orig.set_index('date')
df.sort_index(inplace=True)
df

In [ ]:
ax = df_orig.plot(x='date', y='temp', figsize=(12,6))
xcoords = ['1981-01-01', '1982-01-01','1983-01-01', '1984-01-01', '1985-01-01', '1986-01-01',
          '1987-01-01', '1988-01-01', '1989-01-01', '1990-01-01', '1991-01-01']
for xc in xcoords:
    plt.axvline(x=xc, color='black', linestyle='--')
ax.set_ylabel('temperature')

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(df, marker='.', linestyle='-', linewidth=0.5, label='Weekly')
ax.plot(df.resample('M').mean(), marker='o', markersize=8, linestyle='-', label='Monthly Mean Resample')
ax.set_ylabel('Orders')
ax.legend();

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(df, marker='.', linestyle='-', linewidth=0.5, label='Weekly')
ax.plot(df.resample('Y').mean(), marker='o', markersize=8, linestyle='-', label='Monthly Mean Resample')
ax.set_ylabel('Orders')
ax.legend();

### Decomposing the data

In [ ]:
def seasonal_decompose (df):
    decomposition = sm.tsa.seasonal_decompose(df, model='additive', freq=365)
    
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid
    
    fig = decomposition.plot()
    fig.set_size_inches(14, 7)
    plt.show()
    
    return trend, seasonal, residual

In [ ]:
seasonal_decompose(df)

In [ ]:
def seasonal_decompose (df):
    decomposition = sm.tsa.seasonal_decompose(df.asfreq('MS'), model='additive', freq=12)
    
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid
    
    fig = decomposition.plot()
    fig.set_size_inches(14, 7)
    plt.show()
    
    return trend, seasonal, residual

In [ ]:
seasonal_decompose(df)

### Checking Stationarity

#### Visualization: Graphing the rolling statistics

In [ ]:
def analyze_stationarity(timeseries, title):
    rolmean = pd.Series(timeseries).rolling(window=30).mean() 
    rolstd = pd.Series(timeseries).rolling(window=30).std()
    
    fig, ax = plt.subplots(figsize=(16, 4))
    ax.plot(timeseries, label= title)
    ax.plot(rolmean, label='rolling mean');
    ax.plot(rolstd, label='rolling std (x10)');
    ax.legend()

In [ ]:
pd.options.display.float_format = '{:.8f}'.format
analyze_stationarity(df['temp'], 'raw data')

#### Augmented Dickey-Fuller Test

https://www.statology.org/dickey-fuller-test-python/

- H0: The time series is non-stationary. In other words, it has some time-dependent structure and does not have constant variance over time.
- H1: The time series is stationary.

KPSS is another test for checking the stationarity of a time series. The null and alternate hypothesis for the KPSS test are opposite that of the ADF test.
- H0: The process is trend stationary.
- H1: The series has a unit root (series is not stationary).

In [ ]:
def ADF_test(timeseries):
    print("Results of Dickey-Fuller Test:")
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(
        dftest[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in dftest[4].items():
        dfoutput["Critical Value (%s)" % key] = value
    print(dfoutput)

In [ ]:
ADF_test(df)

In [ ]:
def KPSS_test(timeseries):
    print("Results of KPSS Test:")
    kpsstest = kpss(timeseries.dropna(), regression="c", nlags="auto")    
    kpss_output = pd.Series(
        kpsstest[0:3], index=["Test Statistic", "p-value", "Lags Used"]
    )
    for key, value in kpsstest[3].items():
        kpss_output["Critical Value (%s)" % key] = value
    print(kpss_output)

In [ ]:
KPSS_test(df)

In [ ]:
# Checking stationarity with linear time-series
y_test = df.copy()
tmp = []
for i in range(len(y_test)):
    tmp.append(i)
y_test['temp'] = tmp
ADF_test(y_test)

In [ ]:
# Checking stationarity with sinusoidal data
y_test = df.copy()
tmp = []
for i in range(len(y_test)):
    tmp.append(np.sin(np.pi/2 * i / 365 + 1))
y_test['temp'] = tmp
ADF_test(y_test)

### Checking Trending

#### Detrending

In [ ]:
df_detrend = (df - df.rolling(window=365).mean()) / df.rolling(window=365).std()

analyze_stationarity(df_detrend['temp'].dropna(), 'detrended data')
ADF_test(df_detrend.dropna())

#### Differencing

In [ ]:
df_12lag =  df - df.shift(365)

analyze_stationarity(df_12lag['temp'].dropna(), '12 lag differenced data')
ADF_test(df_12lag.dropna())

#### Detrending + Differencing

In [ ]:
df_365lag_detrend =  df_detrend - df_detrend.shift(365)

analyze_stationarity(df_365lag_detrend['temp'].dropna(), '12 lag differenced de-trended data')
ADF_test(df_365lag_detrend.dropna())

### Create Training & Testing Datasets

In [ ]:
y = df['temp']
y_to_train = y[:'1987-12-31']
y_to_val = y['1988-01-01':]
predict_date = len(y) - len(y[:'1987-12-31'])

### Forecasting models

#### Simple Exponential Smoothing (SES)

Suitable for time series data without trend or seasonal components

In [ ]:
import numpy as np
from statsmodels.tsa.api import SimpleExpSmoothing 

def ses(y, y_to_train, y_to_test,smoothing_level,predict_date):
    y.plot(marker='o', color='black', legend=True, figsize=(14, 7))

    # specific smoothing level
    fit1 = SimpleExpSmoothing(y_to_train).fit(smoothing_level=smoothing_level, optimized=False)
    fcast1 = fit1.forecast(predict_date).rename(r'$\alpha={}$'.format(smoothing_level))    
    plt.plot(y_to_val.index, fcast1, marker='o', color='blue', label=f'smoothing_level={smoothing_level}')    
    fit1.fittedvalues.plot(marker='o',  color='blue', label=f'smoothing_level={smoothing_level}')
    mse1 = ((fcast1 - y_to_test.values) ** 2).mean()
    print(f'The Root Mean Squared Error of our forecasts with smoothing level of {smoothing_level} is {round(np.sqrt(mse1), 2)}')
    
    # auto optimization
    fit2 = SimpleExpSmoothing(y_to_train).fit()
    fcast2 = fit2.forecast(predict_date).rename(r'$\alpha=%s$'%fit2.model.params['smoothing_level'])
    plt.plot(y_to_val.index, fcast2.values, marker='o', color='green', label=f'smoothing_level=auto')
    fit2.fittedvalues.plot(marker='o', color='green', label=f'smoothing_level=auto')
    
    mse2 = ((fcast2 - y_to_test.values) ** 2).mean()
    print(f'The Root Mean Squared Error of our forecasts with auto optimization is {round(np.sqrt(mse2), 2)}')
    
    plt.legend()
    plt.show()

In [ ]:
ses(y, y_to_train, y_to_val, 0.8, predict_date)

#### Holt’s Linear Trend Method

Suitable for time series data with a trend component but without a seasonal component

In [ ]:
from statsmodels.tsa.api import Holt

def holt(y,y_to_train,y_to_test,smoothing_level,smoothing_slope, predict_date):
    y.plot(marker='o', color='black', legend=True, figsize=(14, 7))
    
    fit1 = Holt(y_to_train).fit(smoothing_level, smoothing_slope, optimized=False)
    fcast1 = fit1.forecast(predict_date).rename("Holt's linear trend")
    mse1 = ((fcast1 - y_to_test.values) ** 2).mean()
    print('The Root Mean Squared Error of Holt''s Linear trend {}'.format(round(np.sqrt(mse1), 2)))
    
    fit1.fittedvalues.plot(marker="o", color='blue')
    plt.plot(y_to_val.index, fcast1, color='blue', marker="o")

    plt.show()

In [ ]:
holt(y, y_to_train, y_to_val, 0.6, 0.2, predict_date)

#### Holt-Winters’ Seasonal Method

Suitable for time series data with trend and/or seasonal components

In [ ]:
from statsmodels.tsa.api import ExponentialSmoothing

def holt_win_sea(y, y_to_train, y_to_test, seasonal_period, predict_date):
    y.plot(marker='o', color='black', legend=True, figsize=(14, 7))
    
    fit1 = ExponentialSmoothing(y_to_train, seasonal_periods=seasonal_period, trend='add', seasonal='add').fit(use_boxcox=True)
    fcast1 = fit1.forecast(predict_date).rename('Additive')
    mse1 = ((fcast1 - y_to_test.values) ** 2).mean()
    print('The Root Mean Squared Error of additive trend, additive seasonal of '+ 
          'period season_length={} and a Box-Cox transformation {}'.format(seasonal_period,round(np.sqrt(mse1), 2)))

    fit2 = ExponentialSmoothing(y_to_train, seasonal_periods = seasonal_period, trend='add', seasonal='add', damped_trend=True).fit(use_boxcox=True)
    fcast2 = fit2.forecast(predict_date).rename('Additive+damped')
    mse2 = ((fcast2 - y_to_test.values) ** 2).mean()
    print('The Root Mean Squared Error of additive damped trend, additive seasonal of '+ 
          'period season_length={} and a Box-Cox transformation {}'.format(seasonal_period,round(np.sqrt(mse2), 2)))

    fit1.fittedvalues.plot(style='--', color='red')
    plt.plot(y_to_val.index, fcast1, color='red', marker="o")
    fit2.fittedvalues.plot(style='--', color='green')
    plt.plot(y_to_val.index, fcast2, color='green', marker="o")

    plt.show()

In [ ]:
y_to_train_pos = y_to_train.copy()
y_to_train_pos[y_to_train_pos.values == 0] = 1
y_to_val_pos = y_to_val.copy()
y_to_val_pos[y_to_val_pos.values == 0] = 1

holt_win_sea(y, y_to_train_pos, y_to_val_pos, 365, predict_date)